In [1]:
import os
trainDirPath = "coco128\\images\\train2017\\"
listOfTrainPhotos = os.listdir(trainDirPath) 

In [14]:
import numpy as np
import cv2 as cv
import pickle

In [26]:
listOfTrainPhotos

['000000000009.jpg',
 '000000000025.jpg',
 '000000000030.jpg',
 '000000000034.jpg',
 '000000000036.jpg',
 '000000000042.jpg',
 '000000000049.jpg',
 '000000000061.jpg',
 '000000000064.jpg',
 '000000000071.jpg',
 '000000000072.jpg',
 '000000000073.jpg',
 '000000000074.jpg',
 '000000000077.jpg',
 '000000000078.jpg',
 '000000000081.jpg',
 '000000000086.jpg',
 '000000000089.jpg',
 '000000000092.jpg',
 '000000000094.jpg',
 '000000000109.jpg',
 '000000000110.jpg',
 '000000000113.jpg',
 '000000000127.jpg',
 '000000000133.jpg',
 '000000000136.jpg',
 '000000000138.jpg',
 '000000000142.jpg',
 '000000000143.jpg',
 '000000000144.jpg',
 '000000000149.jpg',
 '000000000151.jpg',
 '000000000154.jpg',
 '000000000164.jpg',
 '000000000165.jpg',
 '000000000192.jpg',
 '000000000194.jpg',
 '000000000196.jpg',
 '000000000201.jpg',
 '000000000208.jpg',
 '000000000241.jpg',
 '000000000247.jpg',
 '000000000250.jpg',
 '000000000257.jpg',
 '000000000260.jpg',
 '000000000263.jpg',
 '000000000283.jpg',
 '00000000029

In [42]:
import numpy as np
import cv2 as cv

img = cv.imread('coco128\\images\\train2017\\000000000034.jpg')
gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)

sift = cv.SIFT_create()
kp = sift.detect(gray,None)

img=cv.drawKeypoints(gray,kp,img)
cv.imwrite('sift_keypoints.jpg',img)

True

In [45]:
sift = cv.SIFT_create()

def getIMGDes(imgPATH):
    img = cv.imread(imgPATH)
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    _, des = sift.detectAndCompute(gray,None)
    return des

In [83]:
curArrOfDesc = np.empty([0, 128])

In [85]:
for imagePath in listOfTrainPhotos:
    fullImagePath = trainDirPath + imagePath
    curArrOfDesc = np.concatenate((curArrOfDesc,getIMGDes(fullImagePath)))

In [86]:
curArrOfDesc.shape

(186062, 128)

In [91]:
type("ds")

str

In [101]:
curArrOfDesc

array([[19., 23.,  8., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  3.],
       [ 7., 13., 18., ...,  0.,  0.,  2.],
       ...,
       [ 0.,  0.,  0., ...,  6.,  1.,  2.],
       [41.,  6.,  0., ...,  3.,  0.,  0.],
       [30., 12.,  0., ...,  0.,  0.,  0.]])

In [103]:
from sklearn.cluster import KMeans

kmeans = sklearn.cluster.KMeans(n_clusters=1024)
kmeans.fit(curArrOfDesc)

KMeans(n_clusters=1024)

In [104]:
import pickle

with open("kmeans.pkl", "wb") as f:
    pickle.dump(kmeans, f)

In [22]:
import pickle

with open("kmeans.pkl", "rb") as f:
    kmeans = pickle.load(f)

In [16]:
sift = cv.SIFT_create()

def getHistoOfIMG(imgPATH):
    res = np.zeros(1024)
    img = cv.imread(imgPATH)
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    pkeys, descs = sift.detectAndCompute(gray,None)
    if len(pkeys) == 0:
        return np.array([])
    kmeansPredict = kmeans.predict(descs.astype(float))
    tmp = np.histogram(kmeansPredict, bins=1024)[0]
    return tmp/np.linalg.norm(tmp)

In [20]:
import os
basePhotoDIR = "VOCdevkit\\VOC2012\\JPEGImages"
listbasePhotoDIR = os.listdir(basePhotoDIR) 
cwd = os.getcwd()

In [26]:
idd = 0
table = []
for nameOfImage in listbasePhotoDIR: 
    path = cwd + "\\" + basePhotoDIR + "\\" + nameOfImage
    vec = getHistoOfIMG(path)
    if len(vec) == 0:
        continue
    dicte = {"id": idd, "path":path, "vector":vec}
    idd+=1
    table.append(dicte)

In [44]:
a = table[0]

In [36]:
a = a.copy()

In [38]:
a['vector'] = [a['vector']]

In [45]:
table

[{'id': 0,
  'path': 'C:\\Users\\Tigran\\Desktop\\cv_project\\VOCdevkit\\VOC2012\\JPEGImages\\2007_000027.jpg',
  'vector': array([0.04416453, 0.04416453, 0.04416453, ..., 0.        , 0.        ,
         0.0331234 ])},
 {'id': 1,
  'path': 'C:\\Users\\Tigran\\Desktop\\cv_project\\VOCdevkit\\VOC2012\\JPEGImages\\2007_000032.jpg',
  'vector': array([0.1628224 , 0.        , 0.        , ..., 0.        , 0.        ,
         0.03256448])},
 {'id': 2,
  'path': 'C:\\Users\\Tigran\\Desktop\\cv_project\\VOCdevkit\\VOC2012\\JPEGImages\\2007_000033.jpg',
  'vector': array([0.03438071, 0.10314212, 0.06876142, ..., 0.03438071, 0.06876142,
         0.06876142])},
 {'id': 3,
  'path': 'C:\\Users\\Tigran\\Desktop\\cv_project\\VOCdevkit\\VOC2012\\JPEGImages\\2007_000039.jpg',
  'vector': array([0.03464795, 0.06929589, 0.        , ..., 0.        , 0.        ,
         0.06929589])},
 {'id': 4,
  'path': 'C:\\Users\\Tigran\\Desktop\\cv_project\\VOCdevkit\\VOC2012\\JPEGImages\\2007_000042.jpg',
  'vecto

In [29]:
len(table[0]['vector'])

1024

In [55]:
df = pd.DataFrame(table)

In [54]:
import sys
np.set_printoptions(threshold=sys.maxsize)


In [83]:
df.to_csv("base.csv", index=False, float_format='%:.17f', sep=',')

In [84]:
import pandas as pd
df2 = pd.read_csv('base.csv')  

In [85]:
df2

id  \
0          0   
1          1   
2          2   
3          3   
4          4   
...      ...   
17119  17119   
17120  17120   
17121  17121   
17122  17122   
17123  17123   

                                                                                  path  \
0      C:\Users\Tigran\Desktop\cv_project\VOCdevkit\VOC2012\JPEGImages\2007_000027.jpg   
1      C:\Users\Tigran\Desktop\cv_project\VOCdevkit\VOC2012\JPEGImages\2007_000032.jpg   
2      C:\Users\Tigran\Desktop\cv_project\VOCdevkit\VOC2012\JPEGImages\2007_000033.jpg   
3      C:\Users\Tigran\Desktop\cv_project\VOCdevkit\VOC2012\JPEGImages\2007_000039.jpg   
4      C:\Users\Tigran\Desktop\cv_project\VOCdevkit\VOC2012\JPEGImages\2007_000042.jpg   
...                                                                                ...   
17119  C:\Users\Tigran\Desktop\cv_project\VOCdevkit\VOC2012\JPEGImages\2012_004326.jpg   
17120  C:\Users\Tigran\Desktop\cv_project\VOCdevkit\VOC2012\JPEGImages\2012_004328.jpg   
17121  C:\Users\Tigran\Desktop\cv_project\VOCdevkit\VOC2012\JPEGImages\2012_004329.jpg   
17122  C:\Users\Tigran\Desktop\cv_project\VOCdevkit\VOC2012\JPEGImages\2012_004330.jpg   
17123  C:\Users\Tigran\Desktop\cv_project\VOCdevkit\VOC2012\JPEGImages\2012_004331.jpg   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [91]:
df2['vector'] =df2['vector'].apply(lambda x: np.asarray(x.replace('\n', '').replace('[','').replace(']','').split()).astype(float))

In [86]:
np.asarray(df2['vector'][0].replace('\n', '').replace('[','').replace(']','').split()).astype(float)

array([0.04416453, 0.04416453, 0.04416453, 0.02208227, 0.05520567,
       0.0331234 , 0.01104113, 0.02208227, 0.0331234 , 0.01104113,
       0.0331234 , 0.0331234 , 0.        , 0.01104113, 0.0331234 ,
       0.02208227, 0.0331234 , 0.0331234 , 0.04416453, 0.0331234 ,
       0.0331234 , 0.01104113, 0.02208227, 0.02208227, 0.01104113,
       0.        , 0.04416453, 0.01104113, 0.02208227, 0.        ,
       0.04416453, 0.05520567, 0.01104113, 0.01104113, 0.02208227,
       0.01104113, 0.0331234 , 0.04416453, 0.08832906, 0.        ,
       0.01104113, 0.0331234 , 0.        , 0.02208227, 0.        ,
       0.02208227, 0.02208227, 0.02208227, 0.0331234 , 0.02208227,
       0.        , 0.0331234 , 0.02208227, 0.01104113, 0.02208227,
       0.02208227, 0.02208227, 0.        , 0.01104113, 0.04416453,
       0.01104113, 0.02208227, 0.01104113, 0.08832906, 0.02208227,
       0.01104113, 0.01104113, 0.05520567, 0.01104113, 0.05520567,
       0.02208227, 0.02208227, 0.04416453, 0.02208227, 0.     

In [201]:
NNmodel.kneighbors([getHistoOfIMG("image.png")])

(array([[0.36847435]]), array([[2382]], dtype=int64))

In [10]:
pd.set_option('display.max_colwidth', None)

In [202]:
print(df[df['id'] == 2382]['path'])

2382    C:\Users\Tigran\Desktop\cv_project\VOCdevkit\VOC2012\JPEGImages\2008_003290.jpg
Name: path, dtype: object


In [1]:
import streamlit as st

st.write("")

2022-12-19 20:50:41.759 
  command:

    streamlit run C:\Users\Tigran\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [2]:
!streamlit run C:\Users\Tigran\anaconda3\lib\site-packages\ipykernel_launcher.py

^C


In [ ]:
!streamlit run "C:\Users\Tigran\Desktop\cv_project\test.py"

In [100]:
df2['id'] == np.asarray(0).astype(int)

0         True
1        False
2        False
3        False
4        False
         ...  
17119    False
17120    False
17121    False
17122    False
17123    False
Name: id, Length: 17124, dtype: bool